# Generate screen diagrams of main game

```
log on                                        
logname "IridisAlphaTitleScreenInterrupts.txt"
tr exec 0916 0916                             
tr exec 0919 0919                             
tr exec 0923 0923                             
tr exec 0935 0935                             
tr exec 0938 0938                             
tr exec 0944 0944                             
tr exec 0947 0947                             
tr exec 098d 098d                             
command 1 "screenshot \"title\" 2"            
command 2 "screenshot \"title\" 2"            
command 3 "screenshot \"title\" 2"            
command 4 "screenshot \"title\" 2"            
command 5 "screenshot \"title\" 2"            
command 6 "screenshot \"title\" 2"            
command 7 "screenshot \"title\" 2"            
command 8 "screenshot \"title\" 2"            
```

Patch for vice:
```c
Index: src/monitor/monitor.c
===================================================================
--- src/monitor/monitor.c       (revision 42926)
+++ src/monitor/monitor.c       (working copy)
@@ -1105,6 +1105,7 @@
 void mon_screenshot_save(const char* filename, int format)
 {
     const char* drvname;
+    static int counter = 0;
 
     switch (format) {
         case 1:
@@ -1123,7 +1124,12 @@
             drvname = "BMP";
             break;
     }
-    if (screenshot_save(drvname, filename, machine_video_canvas_get(0))) {
+    char counter_string[16];
+    sprintf(counter_string, "%d", counter++);
+
+    const char* uniq_name;
+    uniq_name = util_concat(filename, counter_string, ".png", NULL);
+    if (screenshot_save(drvname, uniq_name, machine_video_canvas_get(0))) {
         mon_out("Failed.\n");
     }
 }

```

In [20]:
routines = {
"6c91": "AnimateStarfield",
"6d4c": "AnimateStarfield-Finish",
"6c26": "PerformMainGameUpdate",
"6c6b": "PerformMainGameUpdate-Finish",
}

routine_names = {
"6c91": "AnimateStarfield",
"6d4c": "AnimateStarfield-Finish",
"6c26": "PerformMainGameUpdate",
"6c6b": "PerformMainGameUpdate-Finish",
}

In [29]:
log_file = "IridisAlphaMainGameInterrupts.txt"
input_file = open(log_file,'r')
curr, curr_count, curr_raster, curr_cycles = "", 0, [], []
prev_program_counter = 0

routine_history = []
lines = input_file.readlines()
c = 0
for i in range(6, len(lines), 5):
    address = lines[i][16:20]
    raster = lines[i][23:26].strip()
    prev_program_counter = int(lines[i+1][70:78])
    program_counter = int(lines[i+4][70:78])
    cycles = program_counter - prev_program_counter
    
    routine = routine_names[address]
    routine_history += [(routine, raster, cycles, f"main_game/main{c}.png")]
    c += 1

In [16]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

def createGraph(img, ylabel, raster, label_size):
    fig, ax = plt.subplots(figsize=(20,20))
    ax.imshow(img, extent=(0,504,312,0))
    ax.set_ylabel("Rasterline Position", fontsize = 40)  

    ax.set_yticks([0,int(raster),312])
    ax.set_xticks([0,504])
    plt.yticks(fontsize=label_size)
    plt.xticks(fontsize=label_size)
    
    ax.set_xlabel(f"\n{ylabel}", fontsize = 45)  
    #ax.set_xlabel(r'\begin{center}X-axis\\*\textit{\small{' + label + r'}}\end{center}')
    return plt


In [32]:
import imageio
from PIL import Image 

for routine, raster, cycles, file_name in routine_history[5000:5400]:
    img = Image.open(file_name)
    background = Image.new('RGB', (504,312), "black")
    background.paste(img, (60,20))

    ylabel = f"{routine} ({cycles} Cycles)"
    plt = createGraph(background, ylabel, raster, label_size=25)
    plt.savefig(f"../src/{file_name}", bbox_inches="tight")
    plt.close()

## Clone a bunch of screenshots but with the borders removed

In [37]:
import imageio
from PIL import Image 

for i, (routine, raster, cycles, file_name) in enumerate(routine_history[5000:5400]):
    img = Image.open(file_name)
    cropped = img.crop((31, 32,353,242))

    cropped.save(f"../src/main_game/one_second/main{i}.png")
    img.close()
    cropped.close()